In [43]:
#n=6, q=3, k=2
######################################################################
the_q=3
the_n=6
the_k=2
the_R = GF(the_q)['x']
the_I= the_R.ideal(x^6 + x + 2)
the_A = QuotientRing(the_R, the_I)


######################################################################
# Embedding
######################################################################
def companion_matrix(poly, space):
#Returns the companion matrix in the given space of matrices for a given polynomial poly
    coefficients = poly.coefficients(sparse=False)
    entries = []
    dimension = poly.degree()
    for i in range(dimension):
        row = [0 for j in range(dimension)]
        if i>0:
            row[i-1] = 1
        row[dimension-1] = -coefficients[i]
        entries = entries + row
    return space.matrix(entries) 

def nonsplit_embedding(compmatrix, q, B):
    # Written for n = 6
    # Returns the set of matrices of the form aI+bM+ ... +fM^5 given the compmatrix M were a, ... , f",
    # are in F_q and are not all 0",
    matrices = []
    for a,b,c,d,e,f in GF(q)^the_n:
        if a!= 0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
            matrices.append(B.inverse()*(a*compmatrix^0+b*compmatrix+c*compmatrix^2+d*compmatrix^3+e*compmatrix^4+f*compmatrix^5)*B)
    return matrices

def basis_to_change_matrix(basis, q):
    #takes in a basis of F_q^n and returns the change of basis matrix C_B->A
    # where A is the standard basis 1,x,x^2,... and B is the given basis
    entries = []
    for element in basis:
        coeffs = [0 for i in range(the_n)]
        start = element.coefficients(sparse = False)
        for i in range(len(start)):
            coeffs[i] = start[i]
        entries.append(coeffs)
    transpose = MatrixSpace(GF(q),the_n,the_n).matrix(entries)
    change_matrix = transpose.transpose()
    return change_matrix


######################################################################
#  Tests
######################################################################
def threefive_test(poly, q, B):
    # Written now just for k=2, n=6 (so poly should have degree n=6)",
    # Tests that all 2x6 matrices where the first nonzero determinant is Delta_35"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c in GF(q)^3:
        entries = [0,0,1,a,0,b,0,0,0,0,1,c]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

def twofive_test(poly, q, B):
    # Written now just for k=2, n=6 (so poly should have degree n=6)",
    # Tests that all 2x6 matrices where the first nonzero determinant is Delta_25"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c,d in GF(q)^4:
        entries = [0,1,a,b,0,c,0,0,0,0,1,d]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

def onefive_test(poly, q, B):
    # Written now just for k=2, n=6 (so poly should have degree n=6)",
    # Tests that all 2x6 matrices where the first nonzero determinant is Delta_25"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c,d,e in GF(q)^5:
        entries = [1,a,b,c,0,d,0,0,0,0,1,e]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

######################################################################
# Various dual basis checks
######################################################################

def gal_gen(q):
    # This function returns the generator of the Galois group of F_q^n over F_q,the function which given t
    # returns t^q
    return lambda t: t^q 

def Galois_group(n, q):
    # Returns a list of all elements of the Galois group of F_q^n over F_q, which is generated by the qth power map
    gen = gal_gen(q)
    elements = [gen]
    current = gen
    def compose_with_gen(function):
        return lambda arg: gen(function(arg))
    for i in range(2,n+1):
        new = compose_with_gen(current)
        elements.append(new)
        current = new
    return elements

def trace_of_field_elem(elem, n, q, A):
    # Returns the trace of a given element of F_q^n. This is defined to be the sum over all automorphisms of
    # the Galois group of F_q^n/F_q of these automorphims applied to the element.
    # The argument A is the field F_q^n, given as a quotient of F_q[x] by an irreducible polynomial
    trace_sum = 0
    Gal_group = Galois_group(n,q)
    for auto in Gal_group:
        trace_sum += auto(elem)
    return A(trace_sum).lift()

def dual_basis_check(basis1, basis2, n, q, A):
    if len(basis1) != len(basis2):
        return False
    
    dim = len(basis1)
    dual = True
    for i in range(dim):
        for j in range(i, dim):
            product = A(basis1[i]*basis2[j]).lift()
            prod_trace = trace_of_field_elem(product, n, q, A)
            if i == j and prod_trace != 1:
                #print(prod_trace, i, j)
                dual = False
            if i!=j and prod_trace != 0:
                #print(prod_trace, i, j)
                dual = False
    return dual

 
#FIX
def weakly_self_dual_basis_check(basis, n, q, A):
    # Returns True or False depending on whether or not the argument basis is a weakly self-dual basis of F_q^n/F_q
    # where F_q^n is given as a quotient of F_q[x] by an irreducible polynomial, which is the argument A
    dim = len(basis)
    weakly_self_dual=False
    
    #tests if basis is dual to the scaled, permuted basis
    for gamma in GF(q):
        for p in Permutations(len(basis)):
            permuted_basis = [A(gamma*basis[p(i)-1]).lift() for i in range(1,n+1)]
            if dual_basis_check(basis, permuted_basis, n, q, A):
                weakly_self_dual = True
    return weakly_self_dual


######################################################################
# Normal elements
######################################################################
normal_elems = []
normal_sd_elems = []
for a,b,c,d,e,f in GF(the_q)^the_n:
    if a!=0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
        generating_poly = a*(x^5).polynomial(GF(the_q))+b*(x^4).polynomial(GF(the_q))+c*(x^3).polynomial(GF(the_q))+d*(x^2).polynomial(GF(the_q))+e*(x^1).polynomial(GF(the_q))+f*(x^0).polynomial(GF(the_q))
        powers = [the_A(generating_poly^(the_q^i)).lift() for i in range(the_n)]
        matrix = basis_to_change_matrix(powers, the_q)
        if matrix.det() != 0:
            normal_elems.append(generating_poly)
        if dual_basis_check(powers, powers, the_n, the_q, the_A):
            normal_sd_elems.append(generating_poly)
            

for p in normal_elems:
    print("Normal:", p)
print("Number normal:", len(normal_elems))
print("Number normal and selfdual:", len(normal_sd_elems))


######################################################################
#  Putting it all together
######################################################################

works = []
for poly in normal_elems:
    powers = [the_A(poly^(the_q^i)).lift() for i in range(the_n)]
    matrix = basis_to_change_matrix(powers,the_q)
    result35 = threefive_test((x^6+x+2).polynomial(GF(the_q)), the_q, matrix)
    result25 = twofive_test((x^6+x+2).polynomial(GF(the_q)), the_q, matrix)
    result15 = onefive_test((x^6+x+2).polynomial(GF(the_q)), the_q, matrix)
    if result15 and result25 and result35:
        print("Normal element that's working:", poly)
        print(matrix)
        print()
        works.append(poly)

print("Number normal working:", len(works))

works_and_selfdual = []
for poly in works:
    bas = [the_A(poly^(the_q^i)).lift() for i in range(the_n)]
    if dual_basis_check(bas, bas, the_n, the_q, the_A):
        works_and_selfdual.append(poly)
        
print("Number normal working and selfdual:", len(works_and_selfdual))
    

Normal: x^5 + x^4
Normal: 2*x^5 + x^4
Normal: x^5 + 2*x^4
Normal: 2*x^5 + 2*x^4
Normal: x^5 + x^3
Normal: 2*x^5 + x^3
Normal: x^5 + 2*x^4 + x^3
Normal: 2*x^5 + 2*x^4 + x^3
Normal: x^5 + 2*x^3
Normal: 2*x^5 + 2*x^3
Normal: x^5 + x^4 + 2*x^3
Normal: 2*x^5 + x^4 + 2*x^3
Normal: x^5 + x^2
Normal: 2*x^5 + x^2
Normal: x^5 + x^4 + x^2
Normal: 2*x^5 + x^4 + x^2
Normal: x^5 + x^4 + x^3 + x^2
Normal: 2*x^5 + x^4 + x^3 + x^2
Normal: x^5 + 2*x^4 + x^3 + x^2
Normal: 2*x^5 + 2*x^4 + x^3 + x^2
Normal: x^5 + 2*x^3 + x^2
Normal: 2*x^5 + 2*x^3 + x^2
Normal: x^5 + 2*x^4 + 2*x^3 + x^2
Normal: 2*x^5 + 2*x^4 + 2*x^3 + x^2
Normal: x^5 + 2*x^2
Normal: 2*x^5 + 2*x^2
Normal: x^5 + 2*x^4 + 2*x^2
Normal: 2*x^5 + 2*x^4 + 2*x^2
Normal: x^5 + x^3 + 2*x^2
Normal: 2*x^5 + x^3 + 2*x^2
Normal: x^5 + x^4 + x^3 + 2*x^2
Normal: 2*x^5 + x^4 + x^3 + 2*x^2
Normal: x^5 + x^4 + 2*x^3 + 2*x^2
Normal: 2*x^5 + x^4 + 2*x^3 + 2*x^2
Normal: x^5 + 2*x^4 + 2*x^3 + 2*x^2
Normal: 2*x^5 + 2*x^4 + 2*x^3 + 2*x^2
Normal: x^5 + x
Normal: 2*x^

Number normal working: 0
Number normal working and selfdual: 0


In [ ]:
the_Poly = (x^6+x+1).polynomial(GF(the_q))
works = []
print("Poly:", the_Poly)
for B in GL(the_n, GF(the_q)):
    if onetwo_test(the_Poly,the_q,B):
        print("OK")
        works.append(B)
        print("B:", B)

In [37]:
#n=6, q=2, k=3
######################################################################
the_q=2
the_n=6
the_k=3
the_R = GF(the_q)['x']
the_I= the_R.ideal(x^6 + x + 1)
the_A = QuotientRing(the_R, the_I)


######################################################################
# Embedding
######################################################################
def companion_matrix(poly, space):
#Returns the companion matrix in the given space of matrices for a given polynomial poly
    coefficients = poly.coefficients(sparse=False)
    entries = []
    dimension = poly.degree()
    for i in range(dimension):
        row = [0 for j in range(dimension)]
        if i>0:
            row[i-1] = 1
        row[dimension-1] = -coefficients[i]
        entries = entries + row
    return space.matrix(entries) 

def nonsplit_embedding(compmatrix, q, B):
    # Written for n = 6
    # Returns the set of matrices of the form aI+bM+ ... +fM^5 given the compmatrix M were a, ... , f",
    # are in F_q and are not all 0",
    matrices = []
    for a,b,c,d,e,f in GF(q)^the_n:
        if a!= 0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
            matrices.append(B.inverse()*(a*compmatrix^0+b*compmatrix+c*compmatrix^2+d*compmatrix^3+e*compmatrix^4+f*compmatrix^5)*B)
    return matrices

#I eliminated n from the input of the function and now everything is UNHAPPY
def basis_to_change_matrix(basis, q):
    #takes in a basis of F_q^n and returns the change of basis matrix C_B->A
    # where A is the standard basis 1,x,x^2,... and B is the given basis
    entries = []
    for element in basis:
        coeffs = [0 for i in range(the_n)]
        start = element.coefficients(sparse = False)
        for i in range(len(start)):
            coeffs[i] = start[i]
        entries.append(coeffs)
    transpose = MatrixSpace(GF(q),the_n,the_n).matrix(entries)
    change_matrix = transpose.transpose()
    return change_matrix


######################################################################
#  Tests
######################################################################
def threefourfive_test(poly, q, B):
    # Written now just for k=3, n=6 (so poly should have degree n=6)",
    # Tests that all 3x6 matrices where the first nonzero determinant is Delta_345"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c in GF(q)^3:
        entries = [0,0,1,0,0,a,0,0,0,1,0,b,0,0,0,0,1,c]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True


def twofourfive_test(poly, q, B):
    # Written now just for k=3, n=6 (so poly should have degree n=6)",
    # Tests that all 3x6 matrices where the first nonzero determinant is Delta_245"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c,d in GF(q)^4:
        entries = [0,1,a,0,0,b,0,0,0,1,0,c,0,0,0,0,1,d]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

def fourfivesix_test(poly, q, B):
    # Written now just for k=3, n=6 (so poly should have degree n=6)",
    # Tests that all 3x6 matrices where the first nonzero determinant is Delta_245"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    entries = [0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1]
    element = mspace.matrix(entries)
    elements_with_specific_minors = set()
    for m in torus_elements:
        result = (element*m).echelon_form()
        minors = result.minors(the_k)
        if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
            elements_with_specific_minors.add(result)    
    if len(elements_with_specific_minors) != 1:
        return False
    return True

######################################################################
# Various dual basis checks
######################################################################

def gal_gen(q):
    # This function returns the generator of the Galois group of F_q^n over F_q,the function which given t
    # returns t^q
    return lambda t: t^q 

def Galois_group(n, q):
    # Returns a list of all elements of the Galois group of F_q^n over F_q, which is generated by the qth power map
    gen = gal_gen(q)
    elements = [gen]
    current = gen
    def compose_with_gen(function):
        return lambda arg: gen(function(arg))
    for i in range(2,n+1):
        new = compose_with_gen(current)
        elements.append(new)
        current = new
    return elements

def trace_of_field_elem(elem, n, q, A):
    # Returns the trace of a given element of F_q^n. This is defined to be the sum over all automorphisms of
    # the Galois group of F_q^n/F_q of these automorphims applied to the element.
    # The argument A is the field F_q^n, given as a quotient of F_q[x] by an irreducible polynomial
    trace_sum = 0
    Gal_group = Galois_group(n,q)
    for auto in Gal_group:
        trace_sum += auto(elem)
    return A(trace_sum).lift()

def dual_basis_check(basis1, basis2, n, q, A):
    if len(basis1) != len(basis2):
        return False
    
    dim = len(basis1)
    dual = True
    for i in range(dim):
        for j in range(i, dim):
            product = A(basis1[i]*basis2[j]).lift()
            prod_trace = trace_of_field_elem(product, n, q, A)
            if i == j and prod_trace != 1:
                #print(prod_trace, i, j)
                dual = False
            if i!=j and prod_trace != 0:
                #print(prod_trace, i, j)
                dual = False
    return dual

 
#FIX
def weakly_self_dual_basis_check(basis, n, q, A):
    # Returns True or False depending on whether or not the argument basis is a weakly self-dual basis of F_q^n/F_q
    # where F_q^n is given as a quotient of F_q[x] by an irreducible polynomial, which is the argument A
    dim = len(basis)
    weakly_self_dual=False
    
    #tests if basis is dual to the scaled, permuted basis
    for gamma in GF(q):
        for p in Permutations(len(basis)):
            permuted_basis = [A(gamma*basis[p(i)-1]).lift() for i in range(1,n+1)]
            if dual_basis_check(basis, permuted_basis, n, q, A):
                weakly_self_dual = True
    return weakly_self_dual

######################################################################
# Normal elements
######################################################################
normal_elems = []
normal_sd_elems = []
for a,b,c,d,e,f in GF(the_q)^the_n:
    if a!=0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
        generating_poly = a*(x^5).polynomial(GF(the_q))+b*(x^4).polynomial(GF(the_q))+c*(x^3).polynomial(GF(the_q))+d*(x^2).polynomial(GF(the_q))+e*(x^1).polynomial(GF(the_q))+f*(x^0).polynomial(GF(the_q))
        powers = [the_A(generating_poly^(the_q^i)).lift() for i in range(the_n)]
        matrix = basis_to_change_matrix(powers, the_q)
        if matrix.det() != 0:
            normal_elems.append(generating_poly)
        if dual_basis_check(powers, powers, the_n, the_q, the_A):
            normal_sd_elems.append(generating_poly)
            

for p in normal_elems:
    print("Normal:", p)
print("Number normal:", len(normal_elems))
print("Number normal and sd:", len(normal_sd_elems))
print()


    

######################################################################
#  Putting it all together
######################################################################
poly = normal_sd_elems[0]
powers = [the_A(poly^(the_q^i)).lift() for i in range(the_n)]
for matrix in GL(the_n, GF(the_q)):
    result456 = fourfivesix_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    result345 = threefourfive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    if result456 and result345:
        print(matrix)


Normal: x^5
Normal: x^5 + x^4
Normal: x^5 + x^3
Normal: x^5 + x^4 + x^3
Normal: x^5 + x^2
Normal: x^5 + x^4 + x^3 + x^2
Normal: x^5 + x^4 + x
Normal: x^5 + x^3 + x
Normal: x^5 + x^2 + x
Normal: x^5 + x^4 + x^2 + x
Normal: x^5 + x^3 + x^2 + x
Normal: x^5 + x^4 + x^3 + x^2 + x
Normal: x^5 + 1
Normal: x^5 + x^4 + 1
Normal: x^5 + x^3 + 1
Normal: x^5 + x^4 + x^3 + 1
Normal: x^5 + x^2 + 1
Normal: x^5 + x^4 + x^3 + x^2 + 1
Normal: x^5 + x^4 + x + 1
Normal: x^5 + x^3 + x + 1
Normal: x^5 + x^2 + x + 1
Normal: x^5 + x^4 + x^2 + x + 1
Normal: x^5 + x^3 + x^2 + x + 1
Normal: x^5 + x^4 + x^3 + x^2 + x + 1
Number normal: 24
Number normal and sd: 12

[0 1 1 1 1 1]
[1 1 1 1 1 0]
[0 1 1 1 1 0]
[0 0 1 1 1 0]
[0 1 0 1 0 1]
[0 1 1 1 0 1]
[0 1 0 0 1 0]
[0 1 1 0 1 0]
[0 1 1 1 0 0]
[0 1 1 1 1 1]
[1 1 1 1 1 0]
[0 0 0 1 0 0]
[0 0 1 1 0 1]
[1 0 1 1 0 1]
[1 1 0 1 0 1]
[1 1 1 1 0 1]
[1 1 1 0 1 1]
[0 1 0 1 0 0]
[1 0 1 0 1 0]
[1 0 1 0 1 1]
[0 1 1 0 1 1]
[0 0 1 0 1 1]
[0 0 0 1 1 1]
[1 0 1 0 0 1]
[0 1 1 0 0 1]
[1 1 1

KeyboardInterrupt: 

In [ ]:
#n=6 q=2 k=3 
#old putting it all together

######################################################################
#  Putting it all together
######################################################################

works = []
for poly in normal_elems:
    powers = [the_A(poly^(the_q^i)).lift() for i in range(the_n)]
    matrix = basis_to_change_matrix(powers,the_q)
    result456 = fourfivesix_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    result345 = threefourfive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    print(poly, result456, result345)

print(len(works))

In [2]:
#n=6, q=2, k=1
######################################################################
the_q=2
the_n=6
the_k=1
the_R = GF(the_q)['x']
the_I= the_R.ideal(x^6 + x + 1)
the_A = QuotientRing(the_R, the_I)


######################################################################
# Embedding
######################################################################
def companion_matrix(poly, space):
#Returns the companion matrix in the given space of matrices for a given polynomial poly
    coefficients = poly.coefficients(sparse=False)
    entries = []
    dimension = poly.degree()
    for i in range(dimension):
        row = [0 for j in range(dimension)]
        if i>0:
            row[i-1] = 1
        row[dimension-1] = -coefficients[i]
        entries = entries + row
    return space.matrix(entries) 

def nonsplit_embedding(compmatrix, q, B):
    # Written for n = 6
    # Returns the set of matrices of the form aI+bM+ ... +fM^5 given the compmatrix M were a, ... , f",
    # are in F_q and are not all 0",
    matrices = []
    for a,b,c,d,e,f in GF(q)^the_n:
        if a!= 0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
            matrices.append(B.inverse()*(a*compmatrix^0+b*compmatrix+c*compmatrix^2+d*compmatrix^3+e*compmatrix^4+f*compmatrix^5)*B)
    return matrices

#I eliminated n from the input of the function and now everything is UNHAPPY
def basis_to_change_matrix(basis, q):
    #takes in a basis of F_q^n and returns the change of basis matrix C_B->A
    # where A is the standard basis 1,x,x^2,... and B is the given basis
    entries = []
    for element in basis:
        coeffs = [0 for i in range(the_n)]
        start = element.coefficients(sparse = False)
        for i in range(len(start)):
            coeffs[i] = start[i]
        entries.append(coeffs)
    transpose = MatrixSpace(GF(q),the_n,the_n).matrix(entries)
    change_matrix = transpose.transpose()
    return change_matrix


######################################################################
#  Tests
######################################################################
def test(poly, q, B):
    # Written now just for k=1, n=6 (so poly should have degree n=6)",
    # Tests that all 1x6 matrices where there is some nonzero determinant"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a1,a2,a3,a4,a5,a6 in GF(q)^6:
        entries = [a1,a2,a3,a4,a5,a6]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0]==1 and minors[1]==1 and minors[2]==1 and minors[3]==1 and minors[4]==1 and minors[5]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

######################################################################
# Various dual basis checks
######################################################################

def gal_gen(q):
    # This function returns the generator of the Galois group of F_q^n over F_q,the function which given t
    # returns t^q
    return lambda t: t^q 

def Galois_group(n, q):
    # Returns a list of all elements of the Galois group of F_q^n over F_q, which is generated by the qth power map
    gen = gal_gen(q)
    elements = [gen]
    current = gen
    def compose_with_gen(function):
        return lambda arg: gen(function(arg))
    for i in range(2,n+1):
        new = compose_with_gen(current)
        elements.append(new)
        current = new
    return elements

def trace_of_field_elem(elem, n, q, A):
    # Returns the trace of a given element of F_q^n. This is defined to be the sum over all automorphisms of
    # the Galois group of F_q^n/F_q of these automorphims applied to the element.
    # The argument A is the field F_q^n, given as a quotient of F_q[x] by an irreducible polynomial
    trace_sum = 0
    Gal_group = Galois_group(n,q)
    for auto in Gal_group:
        trace_sum += auto(elem)
    return A(trace_sum).lift()

def dual_basis_check(basis1, basis2, n, q, A):
    if len(basis1) != len(basis2):
        return False
    
    dim = len(basis1)
    dual = True
    for i in range(dim):
        for j in range(i, dim):
            product = A(basis1[i]*basis2[j]).lift()
            prod_trace = trace_of_field_elem(product, n, q, A)
            if i == j and prod_trace != 1:
                #print(prod_trace, i, j)
                dual = False
            if i!=j and prod_trace != 0:
                #print(prod_trace, i, j)
                dual = False
    return dual

 
#FIX
def weakly_self_dual_basis_check(basis, n, q, A):
    # Returns True or False depending on whether or not the argument basis is a weakly self-dual basis of F_q^n/F_q
    # where F_q^n is given as a quotient of F_q[x] by an irreducible polynomial, which is the argument A
    dim = len(basis)
    weakly_self_dual=False
    
    #tests if basis is dual to the scaled, permuted basis
    for gamma in GF(q):
        for p in Permutations(len(basis)):
            permuted_basis = [A(gamma*basis[p(i)-1]).lift() for i in range(1,n+1)]
            if dual_basis_check(basis, permuted_basis, n, q, A):
                weakly_self_dual = True
    return weakly_self_dual

######################################################################
# Normal elements
######################################################################
normal_elems = []
normal_sd_elems = []
for a,b,c,d,e,f in GF(the_q)^the_n:
    if a!=0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
        generating_poly = a*(x^5).polynomial(GF(the_q))+b*(x^4).polynomial(GF(the_q))+c*(x^3).polynomial(GF(the_q))+d*(x^2).polynomial(GF(the_q))+e*(x^1).polynomial(GF(the_q))+f*(x^0).polynomial(GF(the_q))
        powers = [the_A(generating_poly^(the_q^i)).lift() for i in range(the_n)]
        matrix = basis_to_change_matrix(powers, the_q)
        if matrix.det() != 0:
            normal_elems.append(generating_poly)
        if dual_basis_check(powers, powers, the_n, the_q, the_A):
            normal_sd_elems.append(generating_poly)
            

print("Number normal:", len(normal_elems))
print("Number normal and sd:", len(normal_sd_elems))
print()


    

######################################################################
#  Putting it all together
######################################################################

thepoly = normal_elems[1]
count = 0
powers = [the_A(thepoly^(the_q^i)).lift() for i in range(the_n)]
for m in GL(the_n, GF(the_q)):
    count = count+1
    if count%1000 == 0:
        print(count)
    if test((x^6+x+1).polynomial(GF(the_q)), the_q, m):
        print(m)
        print()
        


Number normal: 24
Number normal and sd: 12

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


KeyboardInterrupt: 